In [1]:
%matplotlib inline

In [2]:
import keras
import keras_rcnn.datasets.malaria
import keras_rcnn.models
import keras_rcnn.preprocessing
import matplotlib
import numpy
import sklearn.model_selection
import tensorflow
import keras.backend
import keras.engine
import keras.layers

import keras_resnet.models
import keras_rcnn.backend
import keras_rcnn.classifiers
import keras_rcnn.datasets.malaria
import keras_rcnn.layers
import keras_rcnn.preprocessing

Using TensorFlow backend.


In [3]:
configuration = tensorflow.ConfigProto()

configuration.gpu_options.allow_growth = True

configuration.gpu_options.visible_device_list = "0"

session = tensorflow.Session(config=configuration)

keras.backend.set_session(session)

In [4]:
training, test = keras_rcnn.datasets.malaria.load_data()

training, validation = sklearn.model_selection.train_test_split(training)

classes = {"rbc": 1, "not":2}

generator = keras_rcnn.preprocessing.ObjectDetectionGenerator()

validation_data = keras_rcnn.preprocessing.ObjectDetectionGenerator()

generator = generator.flow(training, classes, (512, 512), 1.0)

validation_data = validation_data.flow(validation, classes, (512, 512), 1.0)

## Training

In [5]:
image = keras.layers.Input((None, None, 3))

model = keras_rcnn.models.RCNN(image, classes=3)

optimizer = keras.optimizers.Adam(0.001)

model.compile(optimizer)

/storage/com/github/broadinstitute/keras-rcnn/keras_rcnn/models.py:178: UserWarning: Output "object_detection_1" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "object_detection_1" during training.
  super(RCNN, self).compile(optimizer, None)
/storage/com/github/broadinstitute/keras-rcnn/keras_rcnn/models.py:178: UserWarning: Output "proposal_target_1" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "proposal_target_1" during training.
  super(RCNN, self).compile(optimizer, None)


In [ ]:
callbacks = [
    keras.callbacks.CSVLogger("./keras-rcnn-training-0.001.csv"),
    keras.callbacks.ModelCheckpoint("./keras-rcnn-checkpoint-0.001.hdf5", monitor="loss")
]

model.fit_generator(
    epochs=10,
    generator=generator,
    steps_per_epoch=512,
    validation_data=validation_data, 
    validation_steps=512
)

/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
511/512 [============================>.] - ETA: 0s - loss: 0.9900

In [ ]:
model.load_weights("./keras-rcnn-checkpoint-0.001.hdf5")

In [ ]:
x = generator.next()[0]

%time y_bounding_boxes, y_proposals, y_scores = model.predict(x)

y_scores = numpy.argmax(y_scores[0], -1)

batch_index = 0

# remove background bounding boxes
y_bounding_boxes = y_bounding_boxes[:, :, 4:]

class_index = 1

a = y_bounding_boxes[batch_index, numpy.where(y_scores == class_index)]

m, n = class_index * 4, (class_index * 4) + 4

a = a[:, :, m:n]

class_index = 2

b = y_bounding_boxes[batch_index, numpy.where(y_scores == class_index)]

m, n = class_index * 4, (class_index * 4) + 4

b = b[:, :, m:n]

In [ ]:
fig, ax = matplotlib.pyplot.subplots(1)

oboxes = []

ax.imshow(x[1][0])

for ii in range(0, a.shape[1]):
    rect = matplotlib.patches.Rectangle(
        (
            a[0][ii][0],
            a[0][ii][1]
        ), 
        a[0][ii][2] - a[0][ii][0], 
        a[0][ii][3] - a[0][ii][1], 
        linewidth=1,
        edgecolor='g',
        facecolor='none'
    )

    oboxes.append(rect)

    pc = matplotlib.collections.PatchCollection(oboxes)

    ax.add_patch(rect)

matplotlib.pyplot.show()